<h2>--- Day 15: Science for Hungry People ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2015FSharp/master?urlpath=lab%2Ftree%2FDay15.ipynb)

<p>Today, you set out on the task of perfecting your milk-dunking cookie recipe.  All you have to do is find the right balance of ingredients.</p>
<p>Your recipe leaves room for exactly <code>100</code> teaspoons of ingredients.  You make a list of the <em>remaining ingredients you could use to finish the recipe</em> (your puzzle input) and their <em>properties per teaspoon</em>:</p>
<ul>
<li><code>capacity</code> (how well it helps the cookie absorb milk)</li>
<li><code>durability</code> (how well it keeps the cookie intact when full of milk)</li>
<li><code>flavor</code> (how tasty it makes the cookie)</li>
<li><code>texture</code> (how it improves the feel of the cookie)</li>
<li><code>calories</code> (how many calories it adds to the cookie)</li>
</ul>
<p>You can only measure ingredients in whole-teaspoon amounts accurately, and you have to be accurate so you can reproduce your results in the future.  The <em>total score</em> of a cookie can be found by adding up each of the properties (negative totals become <code>0</code>) and then multiplying together everything except calories.</p>
<p>For instance, suppose you have <span title="* I know what your preference is, but...">these two ingredients</span>:</p>
<pre><code>Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
</code></pre>
<p>Then, choosing to use <code>44</code> teaspoons of butterscotch and <code>56</code> teaspoons of cinnamon (because the amounts of each ingredient must add up to <code>100</code>) would result in a cookie with the following properties:</p>
<ul>
<li>A <code>capacity</code> of <code>44*-1 + 56*2 = 68</code></li>
<li>A <code>durability</code> of <code>44*-2 + 56*3 = 80</code></li>
<li>A <code>flavor</code> of <code>44*6 + 56*-2 = 152</code></li>
<li>A <code>texture</code> of <code>44*3 + 56*-1 = 76</code></li>
</ul>
<p>Multiplying these together (<code>68 * 80 * 152 * 76</code>, ignoring <code>calories</code> for now) results in a total score of  <code>62842880</code>, which happens to be the best score possible given these ingredients.  If any properties had produced a negative total, it would have instead become zero, causing the whole score to multiply to zero.</p>
<p>Given the ingredients in your kitchen and their properties, what is the <em>total score</em> of the highest-scoring cookie you can make?</p>

In [ ]:
let input = File.ReadAllLines @"input/15.txt"

In [ ]:
type Ingredient = {
    Ingredient : string
    Capacity   : int
    Durability : int
    Flavor     : int
    Texture    : int
    Calories   : int
}

In [ ]:
let parse (line: string) = 
    let split1 = line.Split ": "
    let split2 = split1.[1].Split ", "
    {
        Ingredient = split1.[0]
        Capacity   = split2.[0].Split(" ").[1] |> int
        Durability = split2.[1].Split(" ").[1] |> int
        Flavor     = split2.[2].Split(" ").[1] |> int
        Texture    = split2.[3].Split(" ").[1] |> int
        Calories   = split2.[4].Split(" ").[1] |> int
    }

In [ ]:
let ingredients = 
    input
    |> Array.map parse

ingredients

index,Ingredient,Capacity,Durability,Flavor,Texture,Calories
0,Sprinkles,2,0,-2,0,3
1,Butterscotch,0,5,-3,0,3
2,Chocolate,0,0,5,-1,8
3,Candy,0,-1,0,5,8


Create a function that returns a sequence of lists that add up to a total. `n` is the number of ingredients.

In [ ]:
let rec mixtures n total =
    seq {
        let start = 
            match n with
            | 1 -> total
            | _ -> 0

        for i = start to total do
            let left = total - i
            if n - 1 <> 0 then
                for y in (mixtures (n - 1) left) do
                    yield [i] @ y
            else
                yield [i]
    }

In [ ]:
let calcScore ingredients tsps = 
    let orZero i =
        match i with
        | i when i < 0 -> 0
        | _ -> i

    let (capacity, durability, flavor, texture) = 
        Seq.zip tsps ingredients
        |> Seq.map (fun (tsp, ingredient) -> 
            tsp * ingredient.Capacity, 
            tsp * ingredient.Durability, 
            tsp * ingredient.Flavor, 
            tsp * ingredient.Texture)
        |> Seq.fold (fun (c', d', f', t') (c, d, f, t) -> 
            c' + c, 
            d' + d, 
            f' + f, 
            t' + t) (0,0,0,0)
            
    (capacity |> orZero) * (durability |> orZero) * (flavor |> orZero) * (texture |> orZero)

In [ ]:
#!time
mixtures ingredients.Length 100
|> Seq.map (fun i -> calcScore ingredients i)
|> Seq.max

21367368

Wall time: 318.2019ms

<h2 id="part2">--- Part Two ---</h2>

<p>Your cookie recipe becomes wildly popular!  Someone asks if you can make another recipe that has exactly <code>500</code> calories per cookie (so they can use it as a meal replacement).  Keep the rest of your award-winning process the same (100 teaspoons, same ingredients, same scoring system).</p>
<p>For example, given the ingredients above, if you had instead selected <code>40</code> teaspoons of butterscotch and <code>60</code> teaspoons of cinnamon (which still adds to <code>100</code>), the total calorie count would be <code>40*8 + 60*3 = 500</code>.  The total score would go down, though: only <code>57600000</code>, the best you can do in such trying circumstances.</p>
<p>Given the ingredients in your kitchen and their properties, what is the <em>total score</em> of the highest-scoring cookie you can make with a calorie total of <code>500</code>?</p>

In [ ]:
let calcCalories ingredients tsps = 
    Seq.zip tsps ingredients
    |> Seq.map (fun (tsp, ingredient) -> tsp * ingredient.Calories)
    |> Seq.sum

In [ ]:
#!time
mixtures ingredients.Length 100
|> Seq.map (fun i -> (calcScore ingredients i, calcCalories ingredients i))
|> Seq.filter (fun (_, calories) -> calories = 500)
|> Seq.map (fun (score, _) -> score)
|> Seq.max

1766400

Wall time: 1513.4823ms

[Prev](Day14.ipynb) | [Next](Day16.ipynb)